# 建立回歸模型：準備並視覺化數據

## **南瓜的線性回歸 - 第二課**
#### 介紹

現在你已經準備好使用 Tidymodels 和 Tidyverse 開始構建機器學習模型，接下來可以開始對數據提出問題了。在處理數據並應用機器學習解決方案時，了解如何提出正確的問題以充分挖掘數據的潛力是非常重要的。

在本課中，你將學到：

-   如何為模型構建準備數據。

-   如何使用 `ggplot2` 進行數據視覺化。

你需要回答的問題將決定你會使用哪種類型的機器學習算法。而你獲得答案的質量將在很大程度上取決於數據的性質。

讓我們通過一個實際練習來看看這一點。

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>插圖由 @allison_horst 提供</figcaption>


<!--![插圖由 \@allison_horst 提供](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.tw.jpg)<br>插圖由 \@allison_horst 提供-->


## 1. 匯入南瓜數據並召喚 Tidyverse

我們需要以下套件來完成本課程的數據處理：

-   `tidyverse`： [tidyverse](https://www.tidyverse.org/) 是一個[由 R 套件組成的集合](https://www.tidyverse.org/packages)，旨在讓數據科學更快速、更簡單、更有趣！

您可以使用以下指令安裝：

`install.packages(c("tidyverse"))`

以下腳本會檢查您是否已安裝完成此模組所需的套件，並在缺少某些套件時自動為您安裝。


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

現在，讓我們啟動一些套件並載入本課提供的[數據](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv)！


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

使用 `glimpse()` 快速查看後，可以立即發現資料中有空白值，並且混合了字串（`chr`）和數值型資料（`dbl`）。`Date` 是字元型別，還有一個奇怪的欄位叫做 `Package`，其中的資料混合了 `sacks`、`bins` 和其他值。事實上，這份資料有點混亂 😤。

其實，直接獲得一個完全準備好用來建立機器學習模型的資料集並不常見。但別擔心，在這節課中，你將學習如何使用標準的 R 函式庫來準備一份原始資料集 🧑‍🔧。你還會學到各種技術來視覺化資料。📈📊
<br>

> 溫故知新：管道運算子（`%>%`）透過將物件向前傳遞到函式或呼叫表達式中，按邏輯順序執行操作。你可以將管道運算子理解為程式碼中的「然後」。


## 2. 檢查缺失資料

資料科學家經常需要處理的一個常見問題是資料不完整或缺失。R 使用特殊的哨兵值 `NA`（Not Available）來表示缺失或未知的值。

那麼我們如何知道資料框中是否包含缺失值呢？
<br>
-   一個直接的方法是使用 R 的基礎函數 `anyNA`，它會返回邏輯值 `TRUE` 或 `FALSE`


In [ ]:
pumpkins %>% 
  anyNA()

太好了，看起來有一些遺漏的數據！這是一個不錯的起點。

-   另一種方法是使用函數 `is.na()`，它會用邏輯值 `TRUE` 指出哪些單個欄位元素是遺漏的。


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

好的，完成了工作，但像這樣的大型數據框，逐一檢查所有行和列既低效又幾乎不可能😴。

-   一個更直觀的方法是計算每列的缺失值總和：


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

更好！雖然有些資料缺失，但或許對於當前的任務來說並不重要。我們來看看接下來的分析會帶來什麼結果。

> 除了豐富的套件和函數，R 還擁有非常完善的文件說明。例如，可以使用 `help(colSums)` 或 `?colSums` 來了解更多關於該函數的資訊。


## 3. Dplyr：資料操作的語法

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>插圖由 @allison_horst 提供</figcaption>


<!--![插圖由 \@allison_horst 提供](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.tw.png)<br/>插圖由 \@allison_horst 提供-->


[`dplyr`](https://dplyr.tidyverse.org/)，是 Tidyverse 中的一個套件，提供了一套一致的語法，用於數據操作，並包含一系列動詞，幫助您解決最常見的數據處理挑戰。在本節中，我們將探討一些 dplyr 的動詞！  


#### dplyr::select()

`select()` 是套件 `dplyr` 中的一個函數，幫助你選擇要保留或排除的欄位。

為了讓你的資料框更容易操作，可以使用 `select()` 刪除一些欄位，只保留你需要的欄位。

例如，在這個練習中，我們的分析將涉及到 `Package`、`Low Price`、`High Price` 和 `Date` 這些欄位。讓我們選擇這些欄位。


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` 是 `dplyr` 套件中的一個函數，用於創建或修改欄位，同時保留現有的欄位。

`mutate` 的一般結構為：

`data %>%   mutate(new_column_name = what_it_contains)`

讓我們使用 `Date` 欄位來試試 `mutate`，進行以下操作：

1. 將日期（目前是字元類型）轉換為月份格式（這些是美國日期格式，因此格式為 `MM/DD/YYYY`）。

2. 從日期中提取月份到一個新欄位。

在 R 中，[lubridate](https://lubridate.tidyverse.org/) 套件讓處理日期時間數據變得更加容易。因此，讓我們使用 `dplyr::mutate()`、`lubridate::mdy()`、`lubridate::month()`，來實現上述目標。我們可以刪除 Date 欄位，因為在後續操作中不再需要它。


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

哇哦！🤩

接下來，讓我們新增一個名為 `Price` 的新欄位，代表南瓜的平均價格。現在，讓我們取 `Low Price` 和 `High Price` 欄位的平均值來填充新的 Price 欄位。
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

耶！💪

「等等！」你可能在快速瀏覽整個數據集後使用 `View(pumpkins)` 時會說，「這裡有些奇怪的地方！」🤔

如果你查看 `Package` 欄位，南瓜是以許多不同的方式銷售的。有些是以 `1 1/9 蒲式耳` 的單位銷售，有些是以 `1/2 蒲式耳` 的單位銷售，有些是按南瓜個數銷售，有些是按重量（磅）銷售，還有些是以不同寬度的大箱子銷售。

讓我們來驗證一下：


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

太棒了！👏

南瓜似乎很難穩定地稱重，因此我們可以篩選出在 `Package` 欄位中包含 *bushel* 字串的南瓜，並將其放入一個新的資料框 `new_pumpkins` 中。  


#### dplyr::filter() 和 stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html)：建立一個僅包含符合條件的**列**的資料子集，在此例中，`Package` 欄位中包含 *bushel* 字串的南瓜。

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html)：檢測字串中是否存在或不存在某個模式。

[`stringr`](https://github.com/tidyverse/stringr) 套件提供簡單的函數，用於常見的字串操作。


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

您可以看到，我們已經篩選出大約415行左右的數據，這些數據包含了以蒲式耳計算的南瓜。🤩  
<br>


#### dplyr::case_when()

**但是等等！還有一件事要做**

你是否注意到每行的蒲式耳數量不同？你需要將價格標準化，顯示每蒲式耳的價格，而不是每 1 1/9 或 1/2 蒲式耳的價格。是時候進行一些數學運算來標準化了。

我們將使用函數 [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) 根據一些條件來*變更* Price 欄位的值。`case_when` 允許你向量化多個 `if_else()` 語句。


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

現在，我們可以根據蒲式耳的測量來分析每單位的價格。不過，這些對南瓜蒲式耳的研究表明，`了解數據的本質`是多麼地`重要`！

> ✅ 根據 [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) 的說法，蒲式耳的重量取決於農產品的種類，因為這是一種體積的測量方式。例如，「一蒲式耳的番茄應該重56磅……葉類和綠色蔬菜佔據更多空間但重量較輕，所以一蒲式耳的菠菜只有20磅。」這一切都相當複雜！我們不妨省去將蒲式耳轉換為磅的麻煩，直接按蒲式耳定價。不過，這些對南瓜蒲式耳的研究表明，了解數據的本質是多麼重要！

> ✅ 你有注意到按半蒲式耳出售的南瓜非常昂貴嗎？你能猜出原因嗎？提示：小南瓜比大南瓜貴得多，可能是因為每蒲式耳中小南瓜的數量遠多於大南瓜，這是由於一個大的中空派南瓜佔據了更多未使用的空間。


現在，最後為了冒險的樂趣 💁‍♀️，我們將「月份」欄位移到第一個位置，也就是在「套件」欄位之前。

`dplyr::relocate()` 用於更改欄位的位置。


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

做得好！👌 現在你擁有一個乾淨、整潔的數據集，可以用來建立新的回歸模型！  


## 4. 使用 ggplot2 進行數據視覺化

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>資訊圖表由 Dasani Madipalli 製作</figcaption>


<!--![資訊圖表由 Dasani Madipalli 製作](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.tw.png){width="600"}-->

有一句*智慧*的名言是這樣說的：

> 「簡單的圖表比任何其他工具都能為數據分析師帶來更多的信息。」 --- John Tukey

數據科學家的其中一個角色是展示他們所處理數據的質量和特性。為了達到這個目的，他們通常會創建有趣的視覺化圖表，例如散點圖、折線圖和柱狀圖，來展示數據的不同方面。透過這種方式，他們能夠以視覺化的方式展示數據中難以察覺的關係和空隙。

視覺化還能幫助確定最適合數據的機器學習技術。例如，一個看起來沿著直線分布的散點圖表明該數據非常適合進行線性回歸分析。

R 提供了多種繪製圖表的系統，而 [`ggplot2`](https://ggplot2.tidyverse.org/index.html) 是其中最優雅且最具靈活性的一種。`ggplot2` 允許你透過**組合獨立的組件**來構建圖表。

讓我們從一個簡單的散點圖開始，展示 Price 和 Month 欄位。

在這個例子中，我們將從 [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html) 開始，提供一個數據集和美學映射（使用 [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)），然後添加一個層（例如 [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)）來繪製散點圖。


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

這是一個有用的圖表嗎🤷？有什麼讓你感到驚訝的地方嗎？

它並不是特別有用，因為它只是將你的數據以某個月份的點狀分佈顯示出來。
<br>


### **如何讓它更有用？**

為了讓圖表顯示有用的數據，通常需要以某種方式對數據進行分組。例如，在我們的案例中，找出每個月南瓜的平均價格可以提供更多關於數據中隱藏模式的洞察。這引導我們進一步了解 **dplyr** 的功能：

#### `dplyr::group_by() %>% summarize()`

在 R 中，可以輕鬆地使用以下方法進行分組聚合：

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` 將分析單位從整個數據集更改為個別的分組，例如按月分組。

-   `dplyr::summarize()` 創建一個新的數據框，其中包含每個分組變數的一列，以及您指定的每個摘要統計的一列。

例如，我們可以使用 `dplyr::group_by() %>% summarize()` 將南瓜根據 **Month** 列進行分組，然後找出每個月的 **平均價格**。


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

簡潔明瞭！✨

像月份這類的分類特徵，用長條圖來呈現會更合適 📊。負責繪製長條圖的圖層是 `geom_bar()` 和 `geom_col()`。可以查詢 `?geom_bar` 來了解更多資訊。

讓我們來試試吧！


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩這是一個更有用的數據視覺化！看起來南瓜的最高價格出現在九月和十月。這符合你的預期嗎？為什麼符合或不符合？

恭喜你完成了第二課 👏！你已經為模型建構準備好了數據，並通過視覺化發現了更多的洞察！



---

**免責聲明**：  
本文件已使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵資訊，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或錯誤解釋不承擔責任。
